In [22]:
## import packages
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time 
import cv2
# for openvino
## before this should run setupvar.bat for openvino
from PIL import Image
from openvino import inference_engine as ie
from openvino.inference_engine import IENetwork, IEPlugin
import scipy.io as sio


In [23]:
# Plugin initialization for specified device and load extensions library if specified.
plugin_dir = None
model_xml = './vk/fall_detection_shirui-master/ir_model/shirui_saved.xml'
model_bin = './vk/fall_detection_shirui-master/ir_model/shirui_saved.bin'
# Devices: GPU (intel), CPU, MYRIAD
plugin = IEPlugin("CPU", plugin_dirs=plugin_dir)
# Read IR
net = IENetwork(model=model_xml, weights=model_bin)
print(net.inputs)
print(net.outputs)
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
# Load network to the plugin
exec_net = plugin.load(network=net)
del net

# Plugin initialization for specified device and load extensions library if specified.
plugin_dir_f = None
model_xml_f = './vk/fall_detection_shirui-master/ir_model/shirui_featrue_extractor_saved.xml'
model_bin_f = './vk/fall_detection_shirui-master/ir_model/shirui_featrue_extractor_saved.bin'
# Devices: GPU (intel), CPU, MYRIAD
plugin_f = IEPlugin("CPU", plugin_dirs=plugin_dir_f)
# Read IR
net_f = IENetwork(model=model_xml_f, weights=model_bin_f)
print(net_f.inputs)
print(net_f.outputs)
input_blob_f = next(iter(net_f.inputs))
out_blob_f = next(iter(net_f.outputs))
# Load network to the plugin
exec_featrue_extrctor = plugin_f.load(network=net_f)
del net_f



{'input_3': <openvino.inference_engine.ie_api.DataPtr object at 0x00000240D10B4390>}
{'strided_slice/Squeeze_shrink': <openvino.inference_engine.ie_api.DataPtr object at 0x00000240D10B4390>}
{'zero_padding2d_1_input': <openvino.inference_engine.ie_api.DataPtr object at 0x00000240D10B4390>}
{'strided_slice/Squeeze_shrink': <openvino.inference_engine.ie_api.DataPtr object at 0x00000240D10B4390>}


In [24]:
## define functions to predict

def test_video_realtime(exec_featrue_extrctor,flow, ground_truth,mean_file):
    # Load the mean file to subtract to the images
    # flow is of shape(224,224,20,n)
    num_features = 4096
    d = sio.loadmat(mean_file)
    flow_mean = d['image_mean']
    
    flow = flow - np.tile(flow_mean[...,np.newaxis], (1, 1, 1, flow.shape[3]))
    flow = np.transpose(flow, (3, 0, 1, 2)) 
    predictions = np.zeros((flow.shape[0], num_features), dtype=np.float64)
    truth = np.zeros((flow.shape[0], 1), dtype=np.float64)
      
        # Process each stack: do the feed-forward pass
    for i in range(flow.shape[0]):
        pro_Img = (flow[i]).transpose((2, 0, 1))
        prediction = exec_featrue_extrctor.infer(inputs={input_blob_f: np.expand_dims(pro_Img,0)})
        # Access the results and get the index of the highest confidence score
        output_node_name = list(prediction.keys())[-1]
        prediction = prediction[output_node_name]
        predictions[i, ...] = prediction
        truth[i] = ground_truth
        
    return predictions, truth


def real_time_predict(exec_featrue_extrctor,exec_net,flow_data):
    # flow data is of shape (224x,224y,20,n)
    num_features = 4096
    mean_file = './vk/fall_detection_shirui-master/flow_mean.mat'
    save_features = False
    save_plots = True

    do_training = False
    do_testing = True
    compute_metrics = True
    threshold = 0.5
    ground_truth = 0
    num_features = 4096

    # fold_best_model_path = './urfd_fold_5.h5'
    L = 10

    X2,truth = test_video_realtime(exec_featrue_extrctor,flow_data, ground_truth,mean_file)

    predicted = []
    for i in range(len(X2)):
        predicted.append(  exec_net.infer(inputs={input_blob:np.asarray(X2[i])}) )
    return predicted


    

In [25]:

## set up video stream
vs = VideoStream(src = 1).start()
time.sleep(2.0) # wait for video to start
fps = FPS().start()

cur_images = []
stacked_flow = []
while 1:
    # capture and show the frame
    frame = vs.read()
    # print(frame)
    cv2.imshow("frame:",frame)
    
    
    ###### get the img and the flow ############
    # here name frame needs to contain the current frame in the video
    
    # cur_image stores the recent 2 images (in grey reseized format)
    image = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if len(cur_images) < 2:
        cur_images.append(image)
    else:
        cur_images.append(image)
        cur_images.pop(0)
     
    if len(cur_images) == 2:
        optical_flow = cv2.DualTVL1OpticalFlow_create()
        flow = optical_flow.calc(cur_images[0], cur_images[1], None)

        flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)

        # flow contains the current optival flow
        # while stacked flow contains the flow of recent 10 frame
        stacked_flow.append(flow[..., 0])
        stacked_flow.append(flow[..., 1])
        
        if len(stacked_flow) > 20:
            del stacked_flow[0:2]

    ##### here is the openvino part ###
        if len(stacked_flow) == 20:
            flow_data = np.array(stacked_flow)
            flow_data = flow_data.transpose(1,2,0)
            flow_data = (np.array(stacked_flow)).reshape(224,224,20,1)

            cur_predict = real_time_predict(exec_featrue_extrctor,exec_net,flow_data)

            print(cur_predict)

    #### openvino end
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    fps.update()

[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squeeze_shrink': array([0.9999998], dtype=float32)}]
[{'strided_slice/Squ